#### Imports

In [13]:
import pandas as pd 
import re
from pathlib import Path
import create_dataset as cd
import feature_engineering as fe
import os 
from prep import * 
import numpy as np

In [2]:
data_tim = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_copy.csv')
data_sien = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')
data_tim2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_2_copy.csv')
data_sien2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')

data_tim = data_tim.drop(columns = ['Unnamed: 0'])
data_sien = data_sien.drop(columns = ['Unnamed: 0'])

In [3]:
data = cd.combine_start_dfs(data_sien, data_sien2, data_tim, data_tim2)
# change name climber to numerical value
data['name_climber'] = data['name_climber'].map({'sien':0,'tim':1})
# drop weird column (SIEN's FAULT)
data = data.drop(columns = ['Time (s).1'])
# drop climber rating var 
data = cd.drop_climber_rating(data)
# add datetime var 
data = cd.add_datetime(data)
data
# add heart-rate var



# cut falls 
# data = cd.cut_fall(data)

# data = fe.features(data)
# data.grading = data.grading.apply(fe.french_grade_to_num)

# data = create_snippets(data, 20, 5)
# data = change_climb_ID(data)

data

,entry_num,datetime,Time (s),X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),name_climber,grading,num_attempt,fall_top,outdoor,Unnamed: 0
0,15,2023-06-07 16:36:31.299000000,0.202257,0.616807,0.545096,-0.119170,-0.738735,-0.693665,-0.053444,0,4c,1,0,0.0,NaN
1,15,2023-06-07 16:36:31.399774729,0.403552,0.427085,-0.449999,0.075464,-0.314339,-0.378076,0.015612,0,4c,1,0,0.0,NaN
2,15,2023-06-07 16:36:31.500549459,0.604846,0.570240,0.152899,-0.213106,-0.273812,-0.442562,-0.130227,0,4c,1,0,0.0,NaN
3,15,2023-06-07 16:36:31.601324189,0.806141,0.295633,-0.453686,0.244799,-0.260747,-0.114135,-0.019194,0,4c,1,0,0.0,NaN
4,15,2023-06-07 16:36:31.702098919,1.007435,0.206272,-0.927811,-0.512616,-0.160211,-0.226173,-0.211577,0,4c,1,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5186,40,2023-06-14 17:02:20.736479365,62.783481,0.138590,-1.738872,-0.396787,-0.371624,-0.194527,-0.130351,1,5b,1,0,NaN,NaN
5187,40,2023-06-14 17:02:20.938609523,62.984758,-0.756452,-0.380505,0.312611,0.064059,0.111573,-0.053014,1,5b,1,0,NaN,NaN
5188,40,2023-06-14 17:02:21.140739682,63.186036,-0.321295,-0.189666,-0.070279,-0.062600,-0.059749,-0.085358,1,5b,1,0,NaN,NaN
5189,40,2023-06-14 17:02:21.342869841,63.387314,-0.421361,-0.344314,0.504001,0.049513,-0.251242,0.149490,1,5b,1,0,NaN,NaN


#### Heart rate

- time jumps are inconsistent (sometimes 5, sometimes 10)
- extract & map heart rate values within a datetime range to their group -> dict? 
    - iterate through df groups
        - iterate through json file 
        
- fit values to entries in each group (will have some nans)
- interpolate missing values 

In [4]:
import json 

with open('heartrate/heart_rate-2023-06-07 copy.json', 'r') as f1: 
    hr_json1 = json.load(f1)
with open('heartrate/heart_rate-2023-06-14 copy.json', 'r') as f2: 
    hr_json2 = json.load(f2)


# convert entries to datetime obj & put values in list 
hr_data1 = [] 
for heart_entry in hr_json1:
    dt = heart_entry['dateTime']
    hr = heart_entry['value']['bpm']
    hr_data1.append((dt, hr))

hr_data2 = []
for heart_entry in hr_json2:
    dt = heart_entry['dateTime']
    hr = heart_entry['value']['bpm']
    hr_data2.append((dt, hr))
 

In [44]:
fit_datetimes = [x[0] for x in hr_data1]
fit_hr = [x[1] for x in hr_data1]

# convert fitbit times to datetime obj
fit_datetimes = pd.to_datetime(fit_datetimes)

# NOTE: OK! but need to do this for only the datetimes within a certain interval, cuz otherwise it's gonna match each row in the df
# NOTE: maybe try not to merge dfs, but init a df and then do the masks
# NOTE: adapt this to extract heartrate shit then  

# TODO: INIT null HR variable here? 
if not 'heart-rate' in data:
    data.insert(2, 'heart-rate', float("nan"))

# group by climbID & get datetime intervals 
for group_name, group_data in data.groupby("entry_num"):

    # gets all fitbit datetimes within the given interval 
    start_time = group_data.iloc[0]['datetime']
    end_time = group_data.iloc[-1]['datetime']
    filtered = fit_datetimes[(fit_datetimes >= start_time) & (fit_datetimes <= end_time)]

    # TODO: get 
    # map & retrieve closests df datetimes to fitbit datetimes per group 
    closest_datetimes = filtered.map(lambda x: data['datetime'].iloc[(data['datetime'] - x).abs().idxmin()])    
    print(len(filtered))
    print(len(closest_datetimes))

    print('fitbit', filtered)
    print('df', closest_datetimes)

    break 

    # Filter the dataframe based on the datetime interval
    # filtered_df = data[(df['timestamp'] >= start_datetime) & (df['timestamp'] <= end_datetime)]





# Find closest respective dates
# closest_dates = fit_datetimes.map(lambda x: data['datetime'].iloc[(data['datetime'] - x).abs().idxmin()])
# print(closest_dates)


# # Create a DataFrame with the closest respective dates
# df_new = pd.DataFrame({'closest_datetime': closest_dates, 'new_datetime': fit_datetimes})

# # Merge the original DataFrame with the closest respective dates
# merged_df = pd.merge(data, df_new, left_on='datetime', right_on='closest_datetime', how='outer')

# merged_df




32
32
fitbit DatetimeIndex(['2023-06-07 13:12:52', '2023-06-07 13:12:55',
               '2023-06-07 13:12:58', '2023-06-07 13:13:01',
               '2023-06-07 13:13:04', '2023-06-07 13:13:06',
               '2023-06-07 13:13:09', '2023-06-07 13:13:10',
               '2023-06-07 13:13:13', '2023-06-07 13:13:16',
               '2023-06-07 13:13:18', '2023-06-07 13:13:21',
               '2023-06-07 13:13:22', '2023-06-07 13:13:24',
               '2023-06-07 13:13:29', '2023-06-07 13:13:39',
               '2023-06-07 13:13:44', '2023-06-07 13:13:49',
               '2023-06-07 13:13:54', '2023-06-07 13:13:59',
               '2023-06-07 13:14:04', '2023-06-07 13:14:09',
               '2023-06-07 13:14:14', '2023-06-07 13:14:19',
               '2023-06-07 13:14:24', '2023-06-07 13:14:29',
               '2023-06-07 13:14:34', '2023-06-07 13:14:39',
               '2023-06-07 13:14:44', '2023-06-07 13:14:49',
               '2023-06-07 13:14:54', '2023-06-07 13:14:59'],
          

In [6]:
merged_df.to_csv('temp.csv')

In [29]:

new_datetimes = [x[0] for x in hr_data1]
hr = [x[1] for x in hr_data1]

# iterate through climb IDs
for group_name, group_data in data.groupby("entry_num"):
    print(group_name)
    print(group_data.iloc[0]['datetime'])
    print(group_data.iloc[-1]['datetime'])

    new_datetimes = pd.to_datetime(new_datetimes)

    # closest_dates = new_datetimes.map(lambda x: group_data['datetime'].iloc[(group_data['datetime'] - x).abs().idxmin()])


    # iterate through heart data & match entries by closest datetime 
    # for heart_entry in heart_data1:
    #     print(heart_entry)
    

    # print(type(heart_data1[0]['dateTime']))
    # for entry in heart_data1:
        # if entry['dateTime'] in group_data['datetime']:
        #     print('asdfasdf')
    break 
        




1
2023-06-07 13:12:51.871000-02:00
2023-06-07 13:15:00.053000-02:00


In [22]:
import pandas as pd

# Original DataFrame with datetime entries
df_original = pd.DataFrame({
    'datetime_original': pd.to_datetime(['2023-06-19 12:00:00', '2023-06-19 13:30:00', '2023-06-19 14:45:00']),
    'value_original': [1, 2, 3]
})

# New datetime entries
new_datetimes = pd.to_datetime(['2023-06-19 12:15:00', '2023-06-19 13:40:00'])
print(new_datetimes)

# Find closest respective dates
closest_dates = new_datetimes.map(lambda x: df_original['datetime_original'].iloc[(df_original['datetime_original'] - x).abs().idxmin()])

# Create a DataFrame with the closest respective dates
df_new = pd.DataFrame({'closest_datetime': closest_dates, 'new_datetime': new_datetimes})

# Merge the original DataFrame with the closest respective dates
merged_df = pd.merge(df_original, df_new, left_on='datetime_original', right_on='closest_datetime', how='outer')

merged_df



DatetimeIndex(['2023-06-19 12:15:00', '2023-06-19 13:40:00'], dtype='datetime64[ns]', freq=None)
DatetimeIndex(['2023-06-19 12:00:00', '2023-06-19 13:30:00'], dtype='datetime64[ns]', freq=None)


,datetime_original,value_original,closest_datetime,new_datetime
0,2023-06-19 12:00:00,1,2023-06-19 12:00:00,2023-06-19 12:15:00
1,2023-06-19 13:30:00,2,2023-06-19 13:30:00,2023-06-19 13:40:00
2,2023-06-19 14:45:00,3,NaT,NaT


In [2]:
data_tim = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_copy.csv')
data_sien = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')
data_tim2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_tim_2_copy.csv')
data_sien2 = pd.read_csv('/Users/ryonamba/Documents/VU/VU-master_year_1/ML_quantified_self/ML4QS/datasets/raw_data_sien_copy.csv')


data_tim = data_tim.drop(columns = ['Unnamed: 0'])
data_sien = data_sien.drop(columns = ['Unnamed: 0'])




df = cd.combine_start_dfs(data_sien, data_sien2, data_tim, data_tim2)
df['name_climber'] = df['name_climber'].map({'sien':0,'tim':1})
df
df = df.drop(columns = ['Time (s).1'])
df = cd.drop_climber_rating(df)
df = cd.add_datetime(df)
df = cd.cut_fall(df)


df.grading = df.grading.apply(fe.french_grade_to_num)


df


,entry_num,datetime,Time (s),X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),name_climber,grading,num_attempt,fall_top,outdoor,Unnamed: 0
0,15,2023-06-07 16:36:31.299000-02:00,0.202257,0.616807,0.545096,-0.119170,-0.738735,-0.693665,-0.053444,0,4.7,1,0,0.0,NaN
1,15,2023-06-07 16:36:31.399774729-02:00,0.403552,0.427085,-0.449999,0.075464,-0.314339,-0.378076,0.015612,0,4.7,1,0,0.0,NaN
2,15,2023-06-07 16:36:31.500549459-02:00,0.604846,0.570240,0.152899,-0.213106,-0.273812,-0.442562,-0.130227,0,4.7,1,0,0.0,NaN
3,15,2023-06-07 16:36:31.601324189-02:00,0.806141,0.295633,-0.453686,0.244799,-0.260747,-0.114135,-0.019194,0,4.7,1,0,0.0,NaN
4,15,2023-06-07 16:36:31.702098919-02:00,1.007435,0.206272,-0.927811,-0.512616,-0.160211,-0.226173,-0.211577,0,4.7,1,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5186,40,2023-06-14 17:02:20.736479365-02:00,62.783481,0.138590,-1.738872,-0.396787,-0.371624,-0.194527,-0.130351,1,5.3,1,0,NaN,NaN
5187,40,2023-06-14 17:02:20.938609523-02:00,62.984758,-0.756452,-0.380505,0.312611,0.064059,0.111573,-0.053014,1,5.3,1,0,NaN,NaN
5188,40,2023-06-14 17:02:21.140739682-02:00,63.186036,-0.321295,-0.189666,-0.070279,-0.062600,-0.059749,-0.085358,1,5.3,1,0,NaN,NaN
5189,40,2023-06-14 17:02:21.342869841-02:00,63.387314,-0.421361,-0.344314,0.504001,0.049513,-0.251242,0.149490,1,5.3,1,0,NaN,NaN


In [10]:
# code from DMT
# datatypes - NOTE: subject to change (only accelerometer & gyrometer rn)
types = ['id','datetime', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'numeric', 'categorical', 'ordinal', ] 

# nb of values 
n_values = [data[var].count() for var in data]

# nb of unique values
n_unique = [data[var].nunique() for var in data]


# TODO: do once we have the final variables (eg. for grade)
## mean + std 
# mean_std = []

# for var in data: 
#     if var not in ['datetime', 'entry_num']: # TODO 
#         mean = round(data[var].mean(), 2)
#         std = round(data[var].std(), 2) 
#         res = f'{mean} ({std})'
#         mean_std.append(res)
# mean_std.insert(0,'Na')
# mean_std.insert(0,'Na')

# # range of values
# val_range = []
# for var in data:
#     if var != 'id':
#         r = (data[var].min(), data[var].max())
#         if var != 'time':
#             r = (round(r[0]), round(r[1]))
#         else:
#             r = (data['time'].dt.date.min(), data['time'].dt.date.max())
#         val_range.append(r)
#         r = data['time'].dt.date
# val_range.insert(0, 'Na')

# # date range 
# time_range = []
# for var in data: 
#     # if var != 'id':
#     grouped = data.groupby(var)['time'].min()
#     r = (grouped.dt.date.min(), grouped.dt.date.max())
#     time_range.append(r)


# # missing values before time aggregation TODO: do for heartrate only ig
# missing_before_agg = data.isna().sum().tolist()

# # missing values after time aggregation -> fill this in manually... 
# # automatic 14 for each except call & sms 
# # + 13 for activity 
# # + 1 for 'circumplex.arousal','circumplex.valence', 'mood' & -2 cuz there were like 2 days of measurements in the first 14 days? 
# # temp_data = fn.group_data(data, count=False)
# # missing_after_agg = temp_data.isna().sum().tolist()
# # missing_after_agg.insert(0, 0)


# missing_after_agg = [0 for i in range(len(data.columns))]
# for i, var in enumerate(data.columns):
#     if var == 'sms' or var == 'call' or var == 'id' or var == 'time': 
#         continue
#     if var == 'circumplex.arousal' or var == 'circumplex.valence' or var == 'mood':
#         missing_after_agg[i] += 1
#     missing_after_agg[i] += 14
#     if var == 'activity':
#         missing_after_agg[i] += 13



data_table = pd.DataFrame({'Variables': data.columns, 
                    # 'Data types': types, 
                    'number values': n_values, 
                    'number unique values': n_unique, 
                    # 'Mean (std)': mean_std,
                    # 'Value range': val_range,
                    # 'Date range': time_range, 
                    # 'Missing values (before time aggregation)': missing_before_agg,
                    # 'Missing values (after time aggregation)': missing_after_agg
                    })
data_table

,Variables,number values,number unique values
0,entry_num,18134,25
1,datetime,18134,18134
2,Time (s),18134,18134
3,X (m/s^2),18134,18131
4,Y (m/s^2),18134,18086
5,Z (m/s^2),18134,18111
6,Time (s).1,18134,18134
7,X (rad/s),18134,18133
8,Y (rad/s),18134,18134
9,Z (rad/s),18134,18132
